In [ ]:
import pyodbc
import get_conf
from datetime import timedelta, date, datetime
import pandas as pd
import os
import xarray as xr
import dask
import matplotlib.pyplot as plt

conn = pyodbc.connect(get_conf.conn_string() )

## Kaido HF tables

- Agali_II_FluxHFdata
- Palo_ClearCut_PaloclearHFdata
- Palo_Forest_TensoHFdata
- Soontaga_HFdata

### Soontaga Forest station - Soontaga_HFdata

We need to plot the following columns:

      ,[w_Avg] – vertical wind (Z), m/s
      ,[T_Avg] – sonic temperature, 0C
      ,[CO2_dry_Avg] - CO2, ppm
      ,[H2O_dry_Avg] – H2O, mmol/mol

In [ ]:
days_back = 3
include_now = 0
parameter = "CO2_dry_Avg"
station_table = "Palo_ClearCut_PaloclearHFdata"
filename_def = 'three_days_back'


In [ ]:
d3 = timedelta(days=days_back)

normal_shift = 1 if include_now < 1 else 0

dt_today = date.today() - timedelta(days=normal_shift)

a = dt_today.strftime('%Y-%m-%d 23:59:59') if include_now <= 0 else datetime.now().strftime('%Y-%m-%d %H:%M:%S')

b = (dt_today - d3).strftime('%Y-%m-%d 00:00:00')

a_str = a.replace(' ','_').replace(':','.')
b_str = b.replace(' ','_').replace(':','.')

filename_pref = f"{parameter}_{station_table}_{b_str}_{a_str}" if len(filename_def) <= 0 else f"{filename_def}"

outfilename = f"{filename_pref}.png"

print(a)
print(b)
print(outfilename)

In [ ]:
param = parameter
table = station_table
from_ts = b
to_ts = a

sql_query = f"SELECT {param}, TmStamp FROM {table} WHERE TmStamp >= '{from_ts}' AND TmStamp <= '{to_ts}'"

print(sql_query)

df = pd.read_sql_query(sql_query, conn, coerce_float=True, parse_dates=['TmStamp'], chunksize=None, index_col='TmStamp')


In [ ]:
qr = df.quantile([0.01, 0.99])
qr_min = qr.iloc[0,0]
qr_max = qr.iloc[1,0]

iqrx = (qr_max - qr_min)

vmin = qr_min-iqrx*1.5
vmax = qr_max+iqrx*1.5

print(f"vmin {vmin} vmax {vmax}")

In [ ]:
dset = xr.Dataset.from_dataframe(df)

fig, ax = plt.subplots(1,1, figsize=(14,8))
ax.set_ylim(vmin, vmax)
dset[parameter].plot(ax=ax, color='black', lw=0.2)


plt.title(f"{station_table}: {param} | {from_ts} - {to_ts}")

plt.savefig(os.path.join( 'images', outfilename) )

In [ ]:
print(f"finished plotting {outfilename} at {datetime.now().isoformat()}")